In [4]:
import librosa
import os, gc
import numpy as np
import random

from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score

import xgboost as xgb

In [2]:
def extractMfccFeature(filepath):
    try:
        # returns signal and sample rate
        signal, sr = librosa.load(file_path, offset=12, duration=6)
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=12)
        l = mfccs.shape[1]
        mfccs = mfccs[:, int(0.2 * l):int(0.8 * l)]

        mean = mfccs.mean(axis=1)
        covar = np.cov(mfccs, rowvar=1)

        mean.resize(1, mean.shape[0])
        # it returns matrix.. not useful for machine learning algorithms except KNN
        npArray = np.concatenate((mean, covar), axis=0)
        npArray = npArray.flatten()
        return npArray
    except Exception as e:
        print ("Error accured" + str(e))
        return None

In [12]:
%%time
walk_dir = "genres"

Y = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
     'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

all_input_data = []
all_input_class = []

for root, subdirs, files in os.walk(walk_dir):
    print("Walk on dir:",root,'Total File:',len(files))
    for filename in files:
        if filename.endswith('.au'):
            file_path = os.path.join(root, filename)
            ppFileName = file_path.replace(".au",".pp")
            feature = extractMfccFeature(ppFileName)

            all_input_data.append(feature)
            all_input_class.append(Y[filename.split('.')[0]])


Walk on dir: genres Total File: 0
Walk on dir: genres/pop Total File: 100
Walk on dir: genres/blues Total File: 100
Walk on dir: genres/classical Total File: 100
Walk on dir: genres/country Total File: 100
Walk on dir: genres/hiphop Total File: 100
Walk on dir: genres/rock Total File: 100
Walk on dir: genres/reggae Total File: 100
Walk on dir: genres/metal Total File: 100
Walk on dir: genres/disco Total File: 100
Walk on dir: genres/jazz Total File: 100
CPU times: user 9min 58s, sys: 2min 54s, total: 12min 53s
Wall time: 9min 19s


### Shuffling all data

In [15]:
%%time
print("Shuffle input data to get better results")

all_input_shuffledData = np.asarray(all_input_data)
all_input_shuffledClass = np.asarray(all_input_class)

s = np.arange(all_input_shuffledData.shape[0])
np.random.shuffle(s)

all_input_shuffledData = all_input_shuffledData[s]
all_input_shuffledClass = all_input_shuffledClass[s]

print("->all_input_shuffledData:",all_input_shuffledData.shape)
print("->all_input_shuffledClass:",all_input_shuffledClass.shape)

Shuffle input data to get better results
->all_input_shuffledData: (1000, 156)
->all_input_shuffledClass: (1000,)
CPU times: user 7.84 ms, sys: 0 ns, total: 7.84 ms
Wall time: 24.7 ms


### KNN

In [37]:
%%time
kf = KFold(n_splits=5)

guess = []
actual = []
for trainIndex, testIndex in kf.split(all_input_shuffledData):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train,y_train)
    
    actual.extend(y_test)
    guess.extend(knn.predict(X_test))
    
    k_score = knn.score(X_test,y_test)
    
print("KNN Score: ", accuracy_score(actual, guess))   

KNN Score:  0.367
CPU times: user 548 ms, sys: 3.62 ms, total: 551 ms
Wall time: 548 ms


### SVM

In [35]:
%%time
kf = KFold(n_splits=5)

guess = []
actual = []
for trainIndex, testIndex in kf.split(all_input_shuffledData):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    svm1 = svm.LinearSVC(C=10, loss='squared_hinge', penalty='l2', tol=0.00001)
    svm1.fit(X_train,y_train)
    
    actual.extend(y_test)
    guess.extend(svm1.predict(X_test))

print("SVM Score: ", accuracy_score(actual, guess))   

SVM Score:  0.404
CPU times: user 16.2 s, sys: 81.4 ms, total: 16.3 s
Wall time: 16.4 s


### Decision Tree

In [36]:
%%time
kf = KFold(n_splits=5)

guess = []
actual = []

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    dt = DecisionTreeClassifier()
    dt.fit(X_train, y_train)
    
    actual.extend(y_test)
    guess.extend(dt.predict(X_test))
        
print("Decision Tree Score: ", accuracy_score(actual, guess))


Decision Tree Score:  0.401
CPU times: user 969 ms, sys: 5.11 ms, total: 974 ms
Wall time: 1.01 s


### XGBoost

In [33]:
%%time
kf = KFold(n_splits=5)

guess = []
actual = []

for trainIndex, testIndex in kf.split(all_input_data):
    X_train, X_test = all_input_shuffledData[trainIndex], all_input_shuffledData[testIndex]
    y_train, y_test = all_input_shuffledClass[trainIndex], all_input_shuffledClass[testIndex]
    
    xgb_params = {
        'eta': 0.05,
        'max_depth': 5,
        'subsample': 0.9,
        'objective': 'multi:softmax',
        'silent': 1,
        'colsample_bytree': 0.9,
        'num_class':10
    }

    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_test, y_test)
    
    model = xgb.train(xgb_params, d_train, num_boost_round = 240)
    
    actual.extend(y_test)
    guess.extend(model.predict(d_valid))
    
print('XGBoost Accuracy Score:',accuracy_score(actual, guess))

XGBoost Accuracy Score: 0.621
CPU times: user 6min 55s, sys: 1.86 s, total: 6min 57s
Wall time: 2min 47s


In [38]:
# run garbage collector manually
gc.collect()

42